In [8]:
import gzip
import numpy
from collections import defaultdict

In [9]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [10]:
userRatings = defaultdict(dict)
itemRatings = defaultdict(dict)
beta_u = defaultdict(float)
beta_i = defaultdict(float)

In [11]:
data = list(readGz("train.json.gz"))
train_data = data[:len(data)/2]
validation_data = data[len(data)/2:]

for l in train_data:
    user,item = l['reviewerID'],l['itemID']
    userRatings[user][item] = l['rating']
    itemRatings[item][user] = l['rating']
    beta_u[user] = 0.0
    beta_i[item] = 0.0

In [12]:
MSE=[]
lamda = 1
alpha = 0
for iteration in range(1000):
    original_alpha=alpha
    # print"original_alpha = ",original_alpha
    total = 0
    for u in userRatings:
        for i in userRatings[u]:
            temp = userRatings[u][i] - (beta_u[u] + beta_i[i])
            total += temp
    alpha = total/100000

    #print "iteration",iteration,"alpha = ",alpha
    diff = abs(original_alpha-alpha)
    if diff < 1e-100:
        break

    for u in userRatings:
        total = 0
        for i in userRatings[u]:
            total += userRatings[u][i] - alpha - beta_i[i]
        beta_u[u] = total / (lamda + len(userRatings[u].keys()))

    for i in itemRatings:
        total = 0
        for u in itemRatings[i]:
            total += itemRatings[i][u] - alpha - beta_u[u]
        beta_i[i] = total/(lamda+len(itemRatings[i].keys()))

total = 0
for l in validation_data:
    total += (l['rating'] - (alpha + beta_u[l['reviewerID']] + beta_i[l['itemID']])) ** 2
mse = total / 100000
print"lamda=",lamda,"MSE=", mse
    # MSE.append(mse)

lamda= 1 MSE= 1.28154916045


In [13]:
print"min userBeta =", min(beta_u, key=beta_u.get)
print"max userBeta =", max(beta_u, key=beta_u.get)

print"min itemBeta =", min(beta_i, key=beta_i.get)
print"max itemBeta =", max(beta_i, key=beta_i.get)

min userBeta = U052814411
max userBeta = U816486110
min itemBeta = I071368828
max itemBeta = I558325415


In [15]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  predictions.write(u + '-' + i + ',' + str(alpha + beta_u[u] + beta_i[i]) + '\n')

predictions.close()